In [ ]:
!pip install ultralytics -q
from ultralytics import YOLO
import tqdm
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import wandb
wandb.init(mode="disabled")

In [ ]:
# get prediction boxes string according to the submission format ...

def get_prediction_string(boxes, scores, classes):
    pred_strs = []
    for i, score in enumerate(scores):
        single_pred_str = ""
        single_pred_str += str(float(classes[i])) + " " + str(float(score)) + " "
        
        x_center , y_center, width,height = boxes[i]
        x1 = float(x_center) - (float(width) / 2)
        y1 = float(y_center) - (float(height) / 2)
        width = float(width)
        height = float(height)
        #single_pred_str += " ".join(str(float(x)) for x in boxes[i])
        single_pred_str +=  f"{x1} {y1} {width} {height}"
        
        pred_strs.append(single_pred_str)
    ans = ','.join(map(str, pred_strs))
    if len(ans):
        return ans
#     the solution metrics faield in case of a NaN, '' (empty). So, return "0 0 0 0 0 0" for NaN, '' (empty) string
    return "0 0 0 0 0 0"

# get the predcition in id, ImageID, PredictionString_pred foramt ...

def get_prediction_entry(i, filename, boxes, scores, classes):
    return {
        "id": i, # strating from 0 ...
        "ImageID": filename.split('.')[0], # before the extension ...
        "PredictionString_pred": get_prediction_string(boxes, scores, classes)
    }

# Directory path ...
test_directory = "/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/test"

# Load the model ...
model = YOLO('/kaggle/input/journey-by-neuron-modelckpt/best_second_step.pt')

# do the inference ...

def predict_all_files(test_directory):
    predictions = []
    for i,filename in tqdm(enumerate(os.listdir(test_directory))):
        if filename.endswith(".jpg"):
            filepath = os.path.join(test_directory, filename)
            results = model.predict(source=filepath, conf=0.45, verbose=False)
            boxes = results[0].boxes.xywhn
            scores = results[0].boxes.conf
            classes = results[0].boxes.cls
            prediction = get_prediction_entry(i, filename, boxes, scores, classes)
            predictions.append(prediction)
#             to csv format ...
    predictions_df = pd.DataFrame(predictions)
    return predictions_df
# call the inference function ...
a = predict_all_files(test_directory)

In [ ]:
b = pd.read_csv('/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/sample_submission.csv')

In [ ]:
for i in range(len(b)):
    x = b.iloc[i]['ImageID']
    b.iloc[i, 2] = a.iloc[np.where(a == x)[0][0]]['PredictionString_pred']

In [ ]:
b.head()

In [ ]:
b.to_csv('submission_sust_17.csv', index = False)

## The output csv file corresponds to the score of mAP score of 0.6263